In [2]:
import numpy as np
import cv2

## The Summary of GoogLeNet

In [3]:
def googlenet_summary(model_path):
    net = cv2.dnn.readNetFromONNX(model_path)
    if not net.empty():
        print("Net loaded successfully")
        
    # get the types of layers used in the model
    print("\nNet contains:")
    for t in net.getLayerTypes():
        print("\t{} layers of type {}".format(net.getLayersCount(t), t))
        
    layers_ids, in_shapes, out_shapes = net.getLayersShapes([1, 3, 224, 224])
    layers_names = net.getLayerNames()
    
    # get the tensor shapes for the loaded model and specified input shape.
    print("\nNet layers shapes: ")
    for l in range(len(layers_names)):
        in_num, out_num = len(in_shapes[l]), len(out_shapes[l])
        print("  Layer {} has {} inputs and {} outputs".format(layers_names[l],
                                                             in_num, out_num))
        for i in range(in_num):
            print("\tinput #{} has shape {}".format(i, in_shapes[l][i].flatten()))
        for i in range(out_num):
            print("\toutput #{} has shape {}".format(i, out_shapes[l][i].flatten()))
            
    # compute the number of FLOPs
    print("\ngflops: ", net.getFLOPS((1, 3, 224, 224)) * 1e-9)
    
    # report the amount of memory consumed for storing weights and intermediate tensors
    w, b = net.getMemoryConsumption((1, 3, 224, 224))
    print("\nweights (mb):", w * 1e-6, ", blobs (mb):", b * 1e-6)
    
    # perform a forward pass for a mock input:
    blob = cv2.dnn.blobFromImage(np.zeros((224, 224, 3), np.uint8), 1, (224, 224))
    net.setInput(blob)
    net.forward()
    
    # report the total time
    total, timings = net.getPerfProfile()
    tick2ms = 1000 / cv2.getTickFrequency()
    print("\ninference (ms): {:2f}".format(total * tick2ms))
    
    # report the per layer inference time
    layer_names = net.getLayerNames()
    print("\n{: <30} {}".format("LAYER", "TIME (ms)"))
    for (i,t) in enumerate(timings):
        print("{: <30} {:.2f}".format(layer_names[i], t[0] * tick2ms))

In [4]:
model_path = "./model/model.onnx"
googlenet_summary(model_path)

Net loaded successfully

Net contains:
	9 layers of type Concat
	57 layers of type Convolution
	1 layers of type Dropout
	1 layers of type InnerProduct
	2 layers of type LRN
	14 layers of type Pooling
	57 layers of type Relu
	1 layers of type Reshape
	1 layers of type Softmax
	1 layers of type __NetInputLayer__

Net layers shapes: 
  Layer conv1/7x7_s2_1 has 1 inputs and 1 outputs
	input #0 has shape [  1   3 224 224]
	output #0 has shape [  1   3 224 224]
  Layer conv1/7x7_s2_2 has 1 inputs and 1 outputs
	input #0 has shape [  1   3 224 224]
	output #0 has shape [  1  64 112 112]
  Layer pool1/3x3_s2_1 has 1 inputs and 1 outputs
	input #0 has shape [  1  64 112 112]
	output #0 has shape [  1  64 112 112]
  Layer pool1/norm1_1 has 1 inputs and 1 outputs
	input #0 has shape [  1  64 112 112]
	output #0 has shape [ 1 64 56 56]
  Layer conv2/3x3_reduce_1 has 1 inputs and 1 outputs
	input #0 has shape [ 1 64 56 56]
	output #0 has shape [ 1 64 56 56]
  Layer conv2/3x3_reduce_2 has 1 inputs 


inference (ms): 448.510060

LAYER                          TIME (ms)
conv1/7x7_s2_1                 34.65
conv1/7x7_s2_2                 0.00
pool1/3x3_s2_1                 1.84
pool1/norm1_1                  2.69
conv2/3x3_reduce_1             1.82
conv2/3x3_reduce_2             0.00
conv2/3x3_1                    50.28
conv2/3x3_2                    0.00
conv2/norm2_1                  19.69
pool2/3x3_s2_1                 1.56
inception_3a/1x1_1             1.58
inception_3a/1x1_2             0.00
inception_3a/3x3_reduce_1      3.46
inception_3a/3x3_reduce_2      0.00
inception_3a/3x3_1             19.11
inception_3a/3x3_2             0.00
inception_3a/5x5_reduce_1      0.63
inception_3a/5x5_reduce_2      0.00
inception_3a/5x5_1             8.04
inception_3a/5x5_2             0.00
inception_3a/pool_1            17.24
inception_3a/pool_proj_1       1.64
inception_3a/pool_proj_2       0.00
inception_3a/output_1          0.00
inception_3b/1x1_1             8.02
inception_3b/1x1_2       